In [1]:
# Start by importing necessary libraries
import torch
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms
import onnxruntime as ort


TEST_IMAGE_PATH = r"H:\my_files\my_programs\cat_upscaler\datasets\quality_downsample_only_dataset\LR\0ae5d5a5c1bc4c29a04d09543eb5559c.jpg"
ONNX_MODEL_PATH = r'H:\my_files\my_programs\cat_upscaler\640_1280_cat_images_epochs_5.onnx'
INFER_COUNT = 1

onnx_session = ort.InferenceSession(ONNX_MODEL_PATH)

transform = transforms.Compose(
    [
        transforms.ToTensor(),  # Convert image to tensor
    ]
)


def load_image(image_path):
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0)  # Add batch dimension
    image = image.rot90(1, [2, 3])
    return image


def infer(model, input_tensor):
    inputs = {model.get_inputs()[0].name: input_tensor.numpy()}

    outputs = model.run(None, inputs)

    output_tensor = torch.tensor(outputs[0])
    return output_tensor


def get_higher_res_image(lr_image):
    with torch.no_grad():
        hr_image = infer(onnx_session, lr_image)
        return hr_image


def show_image_comparison(lr_image, hr_image):
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(lr_image)
    plt.title("Low-Resolution Input")
    plt.axis("off")
    plt.subplot(1, 2, 2)
    plt.imshow(hr_image)
    plt.title("Super-Resolution Output")
    plt.axis("off")
    plt.show()


def show_image_as_matplotlib(image):
    plt.imshow(image)
    plt.axis("off")
    plt.show()


def upscale_image(image_path, upscale_count, w, h):
    image = load_image(image_path)

    # resize image to w and h
    image = torch.nn.functional.interpolate(
        image, size=(w, h), mode="bicubic", align_corners=False
    )

    lr_image = image

    images = [image]

    for i in range(upscale_count):
        print(f"doing upscale {i+1} / {upscale_count}", end="\r")
        image = get_higher_res_image(image)
        images.append(image)
    print("\nDone upscaling")
    hr_image = image.squeeze(0).clamp(0, 1).permute(1, 2, 0).numpy()
    lr_image = lr_image.squeeze(0).permute(1, 2, 0).numpy()
    return lr_image, hr_image, images


lr_image, hr_image, images = upscale_image(TEST_IMAGE_PATH, INFER_COUNT, 640, 640)
show_image_comparison(lr_image, hr_image)
for image in images:
    show_image_as_matplotlib(image.squeeze(0).permute(1, 2, 0).numpy())


InvalidArgument: [ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Got invalid dimensions for input: input.1 for the following indices
 index: 2 Got: 640 Expected: 1280
 index: 3 Got: 640 Expected: 1280
 Please fix either the inputs/outputs or the model.